In [82]:
#import libs

import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

sns.set()

In [83]:
#load the data

df = pd.read_csv('train.csv')
df = df.drop('Id', axis=1)

test_data = pd.read_csv('test.csv')
id_ar = test_data['Id']
test_data = test_data.drop('Id', axis=1)

df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [84]:
train, train_target = df.iloc[:, :-1], df.iloc[:, -1]
train_data = pd.concat((train, test_data))

In [85]:
#deal with nan values and standarize the numeric features

for i in train_data.columns:
    if(train_data[i].dtype == np.float64 or train_data[i].dtype == np.int64):
        train_data[i].replace(np.nan, train_data[i].mean(), inplace=True)
        train_data[i] = (train_data[i] - train_data[i].mean()) / train_data[i].std()
    

In [86]:
train_data = pd.get_dummies(train_data, dummy_na=True)
train_data.shape

(2919, 331)

In [87]:
#split the data into train and val

train_data, test_data           = train_data[:train.shape[0]], train_data[train.shape[0]:]
train_inputs, validate_inputs   = train_data[:int(.8 * len(train_data))], train_data[int(.8 * len(train_data)):]
train_targets, validate_targets = train_target[:int(.8 * len(train_target))], train_target[int(.8 * len(train_target)):]


In [88]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor

In [89]:
r_reg = RandomForestRegressor(n_estimators=10)
r_reg.fit(train_inputs, train_targets)
r_reg.score(validate_inputs, validate_targets)

0.7806151091501606

In [90]:
d_reg = DecisionTreeRegressor()
d_reg.fit(train_inputs, train_targets)
d_reg.score(validate_inputs, validate_targets)

0.6066053974320322

In [91]:
s_reg = SGDRegressor()
s_reg.fit(train_inputs, train_targets)
s_reg.score(validate_inputs, validate_targets)

0.7089464546109578

In [96]:
#define the model

input_size = train_inputs.shape[1]
output_size = 1
hidden_layer_size = 1000

model = tf.keras.Sequential([
    tf.keras.layers.Dense(input_size,kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='relu')
])

opt = tf.keras.optimizers.Adam(
   learning_rate = 0.0001
)

model.compile(optimizer=opt, loss='mean_absolute_error', metrics=tf.keras.metrics.RootMeanSquaredError())

batch_size = 128
max_epochs = 100
#early_stopping = tf.keras.callbacks.EarlyStopping(patience=1)

model.fit(train_inputs,
          train_targets,
          epochs=max_epochs,
          batch_size=batch_size,
          #callbacks=[early_stopping],
          validation_data=(validate_inputs, validate_targets),
          verbose = 2 
          )  

Epoch 1/100
10/10 - 3s - loss: 180587.2812 - root_mean_squared_error: 197023.7188 - val_loss: 182234.5156 - val_root_mean_squared_error: 199786.7031 - 3s/epoch - 257ms/step
Epoch 2/100
10/10 - 1s - loss: 180538.7031 - root_mean_squared_error: 196978.9375 - val_loss: 182083.3594 - val_root_mean_squared_error: 199646.5938 - 1s/epoch - 120ms/step
Epoch 3/100
10/10 - 1s - loss: 179937.1875 - root_mean_squared_error: 196422.2656 - val_loss: 180087.5000 - val_root_mean_squared_error: 197802.6562 - 1s/epoch - 143ms/step
Epoch 4/100
10/10 - 1s - loss: 173106.3125 - root_mean_squared_error: 190215.0625 - val_loss: 158395.7188 - val_root_mean_squared_error: 178051.3281 - 1s/epoch - 126ms/step
Epoch 5/100
10/10 - 1s - loss: 112981.0234 - root_mean_squared_error: 140243.4531 - val_loss: 62964.5820 - val_root_mean_squared_error: 80527.1094 - 1s/epoch - 120ms/step
Epoch 6/100
10/10 - 1s - loss: 62459.7461 - root_mean_squared_error: 79624.8750 - val_loss: 53880.7578 - val_root_mean_squared_error: 827

KeyboardInterrupt: 

In [ ]:
validate_targets

1168    235000
1169    625000
1170    171000
1171    163000
1172    171900
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 292, dtype: int64

In [ ]:
test_prediction = model.predict(test_data)

In [ ]:
test_prediction

array([[116876.91],
       [136571.98],
       [176914.42],
       ...,
       [172652.77],
       [108372.91],
       [223158.64]], dtype=float32)

In [ ]:

df = pd.DataFrame(columns=['Id', 'SalePrice'])
df['Id'] = id_ar.values
df['SalePrice'] = test_prediction.squeeze()

In [ ]:
df.to_csv('submission5.csv', index=False)